# Model hidden

In [1]:
"""
Flockers
=============================================================
A Mesa implementation of Craig Reynolds's Boids flocker model.
Uses numpy arrays to represent vectors.
"""

import mesa
import numpy as np


class Boid(mesa.Agent):
    """
    A Boid-style flocker agent.

    The agent follows three behaviors to flock:
        - Cohesion: steering towards neighboring agents.
        - Separation: avoiding getting too close to any other agent.
        - Alignment: try to fly in the same direction as the neighbors.

    Boids have a vision that defines the radius in which they look for their
    neighbors to flock with. Their speed (a scalar) and direction (a vector)
    define their movement. Separation is their desired minimum distance from
    any other Boid.
    """

    def __init__(
        self,
        unique_id,
        model,
        speed,
        direction,
        vision,
        separation,
        cohere=0.03,
        separate=0.015,
        match=0.05,
    ):
        """
        Create a new Boid flocker agent.

        Args:
            speed: Distance to move per step.
            direction: numpy vector for the Boid's direction of movement.
            vision: Radius to look around for nearby Boids.
            separation: Minimum distance to maintain from other Boids.
            cohere: the relative importance of matching neighbors' positions
            separate: the relative importance of avoiding close neighbors
            match: the relative importance of matching neighbors' headings
        """
        super().__init__(unique_id, model)
        self.speed = speed
        self.direction = direction
        self.vision = vision
        self.separation = separation
        self.cohere_factor = cohere
        self.separate_factor = separate
        self.match_factor = match
        self.neighbors = None

    def step(self):
        """
        Get the Boid's neighbors, compute the new vector, and move accordingly.
        """

        self.neighbors = self.model.space.get_neighbors(self.pos, self.vision, False)
        n = 0
        match_vector, separation_vector, cohere = np.zeros((3, 2))
        for neighbor in self.neighbors:
            n += 1
            heading = self.model.space.get_heading(self.pos, neighbor.pos)
            cohere += heading
            if self.model.space.get_distance(self.pos, neighbor.pos) < self.separation:
                separation_vector -= heading
            match_vector += neighbor.direction
        n = max(n, 1)
        cohere = cohere * self.cohere_factor
        separation_vector = separation_vector * self.separate_factor
        match_vector = match_vector * self.match_factor
        self.direction += (cohere + separation_vector + match_vector) / n
        self.direction /= np.linalg.norm(self.direction)
        new_pos = self.pos + self.direction * self.speed
        self.model.space.move_agent(self, new_pos)


class BoidFlockers(mesa.Model):
    """
    Flocker model class. Handles agent creation, placement and scheduling.
    """

    def __init__(
        self,
        seed=None,
        population=100,
        width=100,
        height=100,
        vision=10,
        speed=1,
        separation=1,
        cohere=0.03,
        separate=0.015,
        match=0.05,
    ):
        """
        Create a new Flockers model.

        Args:
            population: Number of Boids
            width, height: Size of the space.
            speed: How fast should the Boids move.
            vision: How far around should each Boid look for its neighbors
            separation: What's the minimum distance each Boid will attempt to
                    keep from any other
            cohere, separate, match: factors for the relative importance of
                    the three drives.
        """
        super().__init__(seed=seed)
        self.population = population
        self.vision = vision
        self.speed = speed
        self.separation = separation

        self.space = mesa.space.ContinuousSpace(width, height, True)
        self.factors = {"cohere": cohere, "separate": separate, "match": match}
        self.make_agents()

    def make_agents(self):
        """
        Create self.population agents, with random positions and starting headings.
        """
        for i in range(self.population):
            x = self.random.random() * self.space.x_max
            y = self.random.random() * self.space.y_max
            pos = np.array((x, y))
            direction = np.random.random(2) * 2 - 1
            boid = Boid(
                unique_id=i,
                model=self,
                speed=self.speed,
                direction=direction,
                vision=self.vision,
                separation=self.separation,
                **self.factors,
            )
            self.space.place_agent(boid, pos)

    def step(self):
        self.agents.shuffle().do("step")

# SugarScape model

In [2]:
class SugarscapeCg(mesa.Model):
    """
    Sugarscape 2 Constant Growback
    """

    verbose = True  # Print-monitoring

    def __init__(self, width=50, height=50, initial_population=100):
        """
        Create a new Constant Growback model with the given parameters.

        Args:
            initial_population: Number of population to start with
        """
        super().__init__()

        # Set parameters
        self.width = width
        self.height = height
        self.initial_population = initial_population

        self.grid = mesa.space.MultiGrid(self.width, self.height, torus=False)
        self.datacollector = mesa.DataCollector(
            {"SsAgent": lambda m: len(m.agents_by_type[SsAgent])}
        )

        # Create sugar
        import numpy as np

        sugar_distribution = np.genfromtxt("sugar-map.txt")
        for _, (x, y) in self.grid.coord_iter():
            max_sugar = sugar_distribution[x, y]
            sugar = Sugar(self, max_sugar)
            self.grid.place_agent(sugar, (x, y))

        # Create agent:
        for i in range(self.initial_population):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            sugar = self.random.randrange(6, 25)
            metabolism = self.random.randrange(2, 4)
            vision = self.random.randrange(1, 6)
            ssa = SsAgent(self, False, sugar, metabolism, vision)
            self.grid.place_agent(ssa, (x, y))

        self.running = True
        self.datacollector.collect(self)

    def step(self):
        # Step suger and agents
        self.agents_by_type[Sugar].do("step")
        self.agents_by_type[SsAgent].shuffle_do("step")
        # collect data
        self.datacollector.collect(self)
        if self.verbose:
            print(f"Step: {self.steps}, SsAgents: {len(self.agents_by_type[SsAgent])}")

    def run_model(self, step_count=200):
        if self.verbose:
            print(
                f"Initial number Sugarscape Agents: {len(self.agents_by_type[SsAgent])}"
            )

        for i in range(step_count):
            self.step()

        if self.verbose:
            print(
                f"\nFinal number Sugarscape Agents: {len(self.agents_by_type[SsAgent])}"
            )

In [3]:
def get_distance(pos_1, pos_2):
    """Get the distance between two point

    Args:
        pos_1, pos_2: Coordinate tuples for both points.
    """
    x1, y1 = pos_1
    x2, y2 = pos_2
    dx = x1 - x2
    dy = y1 - y2
    return math.sqrt(dx**2 + dy**2)


class SsAgent(mesa.Agent):
    def __init__(self, model, moore=False, sugar=0, metabolism=0, vision=0):
        super().__init__(model)
        self.moore = moore
        self.sugar = sugar
        self.metabolism = metabolism
        self.vision = vision

    def get_sugar(self, pos):
        this_cell = self.model.grid.get_cell_list_contents([pos])
        for agent in this_cell:
            if type(agent) is Sugar:
                return agent

    def is_occupied(self, pos):
        this_cell = self.model.grid.get_cell_list_contents([pos])
        return any(isinstance(agent, SsAgent) for agent in this_cell)

    def move(self):
        # Get neighborhood within vision
        neighbors = [
            i
            for i in self.model.grid.get_neighborhood(
                self.pos, self.moore, False, radius=self.vision
            )
            if not self.is_occupied(i)
        ]
        neighbors.append(self.pos)
        # Look for location with the most sugar
        max_sugar = max(self.get_sugar(pos).amount for pos in neighbors)
        candidates = [
            pos for pos in neighbors if self.get_sugar(pos).amount == max_sugar
        ]
        # Narrow down to the nearest ones
        min_dist = min(get_distance(self.pos, pos) for pos in candidates)
        final_candidates = [
            pos for pos in candidates if get_distance(self.pos, pos) == min_dist
        ]
        self.random.shuffle(final_candidates)
        self.model.grid.move_agent(self, final_candidates[0])

    def eat(self):
        sugar_patch = self.get_sugar(self.pos)
        self.sugar = self.sugar - self.metabolism + sugar_patch.amount
        sugar_patch.amount = 0

    def step(self):
        self.move()
        self.eat()
        if self.sugar <= 0:
            self.model.grid.remove_agent(self)
            self.remove()


class Sugar(mesa.Agent):
    def __init__(self, model, max_sugar):
        super().__init__(model)
        self.amount = max_sugar
        self.max_sugar = max_sugar

    def step(self):
        self.amount = min([self.max_sugar, self.amount + 1])

# test sugar scape

In [4]:
from pathlib import Path

In [5]:
model = SugarscapeCg(width=50, height=50, initial_population=100)

TypeError: Agent.__init__() missing 1 required positional argument: 'model'

# Testing

In [6]:
model = BoidFlockers(100, 100, 100, speed=5, vision=5, separation=1)


In [7]:
from mesa.cacheable_model import CacheableModel
parameters = ['unique_id', 'pos', 'direction', 'vision', 'speed']
cacheable_model = CacheableModel(model, "output_dir", 100, 100, condition_function=None)
for i in range(100):
    cacheable_model.model.step()
    cacheable_model.get_data(parameters)

0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/grid_data_00.parquet
0
output_dir/g

In [8]:
import pyarrow.parquet as pq

# Specify the path to your Parquet file
parquet_file_path = 'output_dir/grid_data_00.parquet'

# Read the Parquet file
table = pq.read_table(parquet_file_path)

# Convert the table to a pandas DataFrame 
df = table.to_pandas()

# Display the DataFrame
df

,unique_id,pos,direction,vision,speed
0,0,"[35.16097010240313, 19.999474315871907]","[-0.7473243931438508, -0.6644593677661375]",5,5
1,1,"[38.79573096923578, 62.090884941004916]","[0.5488249488082567, -0.8359373036093164]",5,5
2,2,"[61.80900256297662, 39.85568409709933]","[0.8238606748642199, -0.5667923679904945]",5,5
3,3,"[80.81284510937842, 91.15839853728859]","[-0.8183123919426883, -0.574773719991647]",5,5
4,4,"[10.232938049159117, 41.86371728245083]","[0.9403802236406391, -0.34012502846097875]",5,5
...,...,...,...,...,...
95,95,"[34.26374998874098, 57.72817674806151]","[-0.7615181045246203, 0.6481436387724788]",5,5
96,96,"[69.42021067172772, 58.240745328099905]","[-0.8571043819751183, -0.5151427747712769]",5,5
97,97,"[69.89356058116424, 5.911341498177485]","[0.06338689983849959, 0.997989028461167]",5,5
98,98,"[88.72109018952585, 16.14148214819678]","[-0.1807110265957716, -0.9835362346485779]",5,5


In [9]:
# reconstructing the agent

In [10]:
column_list = df.columns

for idx, row in df.iterrows():
    for column in column_list:
        print(row[column])

0
[35.1609701  19.99947432]
[-0.74732439 -0.66445937]
5
5
1
[38.79573097 62.09088494]
[ 0.54882495 -0.8359373 ]
5
5
2
[61.80900256 39.8556841 ]
[ 0.82386067 -0.56679237]
5
5
3
[80.81284511 91.15839854]
[-0.81831239 -0.57477372]
5
5
4
[10.23293805 41.86371728]
[ 0.94038022 -0.34012503]
5
5
5
[13.96351077  4.21115639]
[0.07241175 0.99737482]
5
5
6
[79.83145771 12.5216323 ]
[-0.34469276 -0.93871556]
5
5
7
[47.89253115  7.10641491]
[-0.67980145 -0.73339621]
5
5
8
[61.38948976 66.57486984]
[-0.6971417   0.71693336]
5
5
9
[93.13133623 16.58976691]
[ 0.87415445 -0.48564802]
5
5
10
[67.59005771 25.87480249]
[ 0.35055812 -0.93654098]
5
5
11
[28.51316701 25.89078985]
[-0.70350044 -0.71069483]
5
5
12
[27.22610483 83.38860442]
[0.6490938  0.76070838]
5
5
13
[65.05117909 76.22130824]
[ 0.49807534 -0.86713376]
5
5
14
[22.86589289 76.03178069]
[-0.99268126 -0.12076387]
5
5
15
[17.31652309  2.6388061 ]
[0.13822015 0.99040153]
5
5
16
[39.48337682 94.27979254]
[-0.00999137  0.99995008]
5
5
17
[95.156945

In [11]:
mesa.Model().agents_
model.agents_

defaultdict(dict,
            {__main__.Boid: {<__main__.Boid at 0x7fca50d55810>: None,
              <__main__.Boid at 0x7fca50d55600>: None,
              <__main__.Boid at 0x7fca50d566e0>: None,
              <__main__.Boid at 0x7fca50d55a50>: None,
              <__main__.Boid at 0x7fca50d57070>: None,
              <__main__.Boid at 0x7fca50d56500>: None,
              <__main__.Boid at 0x7fca50d55c90>: None,
              <__main__.Boid at 0x7fca50d562c0>: None,
              <__main__.Boid at 0x7fca50d56620>: None,
              <__main__.Boid at 0x7fca50d56470>: None,
              <__main__.Boid at 0x7fca50d55b40>: None,
              <__main__.Boid at 0x7fca50d55ae0>: None,
              <__main__.Boid at 0x7fca50d55ab0>: None,
              <__main__.Boid at 0x7fca50d560b0>: None,
              <__main__.Boid at 0x7fca50d56260>: None,
              <__main__.Boid at 0x7fca50d55630>: None,
              <__main__.Boid at 0x7fca50d56710>: None,
              <__main__.Boid at 

In [12]:
reconstructed_model = mesa.Model()
column_list = df.columns
agent_list = []
for idx, row in df.iterrows():
    agent = Boid(None, reconstructed_model, None, None, None, None)
    for column in column_list:
        setattr(agent, column, row[column])
    agent_list.append(agent)
agent_list

In [13]:
for i in reconstructed_model.agents:
    print(i)

# Work in Progress

In [14]:
def get_top_level_data(params_list):
    top_level_data = {}
    for param in params_list:
        top_level_data[param] = cacheable_model.model.__dict__[param]
    return top_level_data
get_top_level_data([ 'speed', 'vision',  'separation'])


{'speed': 5, 'vision': 5, 'separation': 1}

In [15]:
#model
model.__dict__

{'_seed': 0.9371898361867775,
 'random': <random.Random at 0x5591affcd950>,
 '_steps': 0,
 '_time': 0,
 'running': True,
 'schedule': None,
 'current_id': 0,
 'agents_': defaultdict(dict,
             {__main__.Boid: {<__main__.Boid at 0x7fca50d55810>: None,
               <__main__.Boid at 0x7fca50d55600>: None,
               <__main__.Boid at 0x7fca50d566e0>: None,
               <__main__.Boid at 0x7fca50d55a50>: None,
               <__main__.Boid at 0x7fca50d57070>: None,
               <__main__.Boid at 0x7fca50d56500>: None,
               <__main__.Boid at 0x7fca50d55c90>: None,
               <__main__.Boid at 0x7fca50d562c0>: None,
               <__main__.Boid at 0x7fca50d56620>: None,
               <__main__.Boid at 0x7fca50d56470>: None,
               <__main__.Boid at 0x7fca50d55b40>: None,
               <__main__.Boid at 0x7fca50d55ae0>: None,
               <__main__.Boid at 0x7fca50d55ab0>: None,
               <__main__.Boid at 0x7fca50d560b0>: None,
             

In [16]:
#agent
agent_dict = list(model.agents_.values())
complete_row_data_deserialized = []
for key, val in agent_dict[0].items():
    complete_row_data_deserialized.append(key.__dict__)
complete_row_data_deserialized

[{'unique_id': 0,
  'model': <__main__.BoidFlockers at 0x7fca50d559f0>,
  'pos': array([35.1609701 , 19.99947432]),
  'speed': 5,
  'direction': array([-0.74732439, -0.66445937]),
  'vision': 5,
  'separation': 1,
  'cohere_factor': 0.03,
  'separate_factor': 0.015,
  'match_factor': 0.05,
  'neighbors': []},
 {'unique_id': 1,
  'model': <__main__.BoidFlockers at 0x7fca50d559f0>,
  'pos': array([38.79573097, 62.09088494]),
  'speed': 5,
  'direction': array([ 0.54882495, -0.8359373 ]),
  'vision': 5,
  'separation': 1,
  'cohere_factor': 0.03,
  'separate_factor': 0.015,
  'match_factor': 0.05,
  'neighbors': []},
 {'unique_id': 2,
  'model': <__main__.BoidFlockers at 0x7fca50d559f0>,
  'pos': array([61.80900256, 39.8556841 ]),
  'speed': 5,
  'direction': array([ 0.82386067, -0.56679237]),
  'vision': 5,
  'separation': 1,
  'cohere_factor': 0.03,
  'separate_factor': 0.015,
  'match_factor': 0.05,
  'neighbors': [<__main__.Boid at 0x7fca50ba9030>]},
 {'unique_id': 3,
  'model': <__ma

In [17]:
def get_agent_data(self, parameters):
    agent_dict = list(self.model.agents_.values())
    complete_row_data_deserialized = []
    for key, val in agent_dict[0].items():
        complete_row_data_deserialized.append(key.__dict__)
    boids_data = []
    for row_record in complete_row_data_deserialized:
        clean_row_data = {}
        for param in parameters:
            clean_row_data[param] = row_record[param]
        boids_data.append(clean_row_data)
    boids_table = pa.Table.from_pylist(boids_data)
    padding = len(str(self._total_steps)) - 1
    print(self.model._steps)
    filename = f"{self.cache_file_path}/grid_data_{self.model._steps:0{padding}}.parquet"
    print(filename)
    pq.write_table(boids_table, filename)
    return boids_data

In [18]:
import matplotlib.pyplot as plt
def draw_boids(model):
    x_vals = []
    y_vals = []
    for boid in model.agents:
        x, y = boid.pos
        x_vals.append(x)
        y_vals.append(y)
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    ax.scatter(x_vals, y_vals)

In [19]:
draw_boids(model_a)

NameError: name 'model_a' is not defined

In [20]:
from mesa.visualization import SolaraViz#, make_space_matplotlib

def boid_draw(agent):
    return {"color": "tab:red"}


model_params = {
    "population": 100,
    "width": 100,
    "height": 100,
    "speed": 5,
    "vision": 10,
    "separation": 2,
}

model = BoidFlockers(100, 100, 100, 5, 10, 2)

page = SolaraViz(
    model_class=BoidFlockers,
    model_params=model_params,
    measures=[],
    name="BoidFlockers",
    agent_portrayal=boid_draw,
)

In [21]:
page

Cannot show ipywidgets in text